# ABBA-BABA Test
In this notebook I performed the ABBA-BABA Test using the new outgroup allele counts <b>(see 09112019_outgroup_allele_count_phase2_dataset_creation)</b> and the new allele count dataset for the phase2.

I have used part of <b>Alistair's notebook (20150529 Admixture, introgression.ipynb)</b> on the Ag1000G github repository

-------------------

Loading modules and callsets paths:

In [1]:
%run imports.ipynb

In [2]:
from dask_kubernetes import KubeCluster
cluster = KubeCluster(n_workers=30)
cluster

In [3]:
from dask.distributed import Client
client = Client(cluster)
client

Client Scheduler: tcp://10.34.231.226:42263 Dashboard: /user/carlo%20mariade%20marco1/proxy/8787/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [4]:
chromosomes = ['3L', '3R', '2L', '2R', 'X']

In [5]:
out_species = ['arab', 'chri', 'epir', 'mela', 'meru', 'quad']

In [6]:
an_col = 'AOcol', 'BFcol', 'GNcol', 'CIcol', 'GHcol'
an_gam = 'GHgam', 'CMgam', 'BFgam', 'GNgam', 'GQgam', 'UGgam', 'GAgam', 'FRgam'
an_na = 'KE', 'GM', 'GW'

In [7]:
calldata_out= zarr.open('data/outgroup_alleles_phase2.zarr')

In [8]:
calldata_biallel= zarr.open('data/phase2_biallel_allele_count.zarr')

In [9]:
metadata = pd.read_csv("samples.meta.txt", sep="\t")
metadata.head()

,ox_code,src_code,population,country,location,site,contributor,contact,year,m_s,sex,n_sequences,mean_coverage,ebi_sample_acc,latitude,longitude
0,AA0040-C,Twifo_Praso__E2,GHcol,Ghana,Twifo Praso,Twifo Praso,David Weetman,David Weetman,2012,M,F,95033368,30.99,ERS311878,5.60858,-1.54926
1,AA0041-C,Twifo_Praso__H3,GHcol,Ghana,Twifo Praso,Twifo Praso,David Weetman,David Weetman,2012,M,F,95843804,31.70,ERS311886,5.60858,-1.54926
2,AA0042-C,Takoradi_C7,GHcol,Ghana,Takoradi,Takoradi,David Weetman,David Weetman,2012,M,F,107420666,35.65,ERS311894,4.91217,-1.77397
3,AA0043-C,Takoradi_H8,GHcol,Ghana,Takoradi,Takoradi,David Weetman,David Weetman,2012,M,F,95993752,29.46,ERS311902,4.91217,-1.77397
4,AA0044-C,Takoradi_D10,GHcol,Ghana,Takoradi,Takoradi,David Weetman,David Weetman,2012,M,F,103044262,33.67,ERS311910,4.91217,-1.77397


In [10]:
ac_cache = dict()

def load_ac(chrom, pop):
    if (chrom, pop) in ac_cache:
        return ac_cache[chrom, pop]
    else:
        if pop in out_species:
            ac = calldata_out[chrom][pop]
        else:
            ac = calldata_biallel[chrom][pop]
            ac_cache[chrom, pop] = ac
        return ac

In [11]:
fourpop_cache = dict()

-----------------------------------------

In [15]:
def f4_analysis(chroms, A, B, C, D, regions=None, blen=100000, plot=False, ax=None):
    
    
    
    region_str = ",".join(["{0}_{1}".format(r.start, r.stop) for r in regions])
    
    key = (",".join(chroms), region_str, A, B, C, D, blen)
    if key in fourpop_cache:
        # re-use from cache
        d, d_se, d_z, d_vb, d_vj = fourpop_cache[key]
        
    else:
        # load allele counts
        aca = dask.array.vstack([load_ac(chrom, A)[region] for chrom, region in zip(chroms, regions)])
        acb = dask.array.vstack([load_ac(chrom, B)[region] for chrom, region in zip(chroms, regions)])
        acc = dask.array.vstack([load_ac(chrom, C)[region] for chrom, region in zip(chroms, regions)])
        acd = dask.array.vstack([load_ac(chrom, D)[region] for chrom, region in zip(chroms, regions)])
            
        # run D test
        d, d_se, d_z, d_vb, d_vj = allel.blockwise_patterson_d(aca, acb, acc, acd, 
                                                                     blen=blen)
        
        # cache for re-use
        fourpop_cache[key] = d, d_se, d_z, d_vb, d_vj
    
    return d, d_se, d_z, d_vb, d_vj

def f4_analys(As, Bs, Cs, Ds, chroms=chromosomes, regions=None, blen=100000):
    
    if regions is None:
        regions = [slice(None)] * 4
    
    region_str = ",".join(["{0}_{1}".format(r.start, r.stop) for r in regions])
    # normalise inputs
    if not isinstance(chroms, (list, tuple)):
        chroms = [chroms]
    if not isinstance(As, (list, tuple)):
        As = [As]
    if not isinstance(Bs, (list, tuple)):
        Bs = [Bs]
    if not isinstance(Cs, (list, tuple)):
        Cs = [Cs]
    if not isinstance(Ds, (list, tuple)):
        Ds = [Ds]
        
    # setup output table
    tbl = [['chromosome', 'test', 'D', 'SE', 'Z']]

    for A in As:
        for B in Bs:
            if A != B:
                for C in Cs:
                    for D in Ds:
                        if C != D:
                            d, d_se, d_z, _, _ = f4_analysis(chroms, A, B, C, D, 
                                                             regions=regions, blen=blen)
                            test = 'D(%s, %s; %s, %s)' % (A, B, C, D)
                            row = [",".join(chroms), test, d, d_se, d_z]
                            tbl += [row]

    # display results
    (etl
     .wrap(tbl)
     .interpolate('D', '%.3f')
     .interpolate('SE', '%.4f')
     .interpolate('Z', '%.1f')
     .displayall(index_header=False, 
                 tr_style=lambda row: 'background-color: %s' % ('#afa' if float(row.Z) > 5 else '#aaf' if float(row.Z) < -5 else 'white')))
    

In [22]:
A = an_gam
B = 'GAgam'
C = an_col
D = 'chri'

In [17]:
f4_analys(A, B, C, D, chroms=("3L", "3R"),
                regions=(slice(15000000, 41000000), slice(1000000, 37000000)))

chromosome,test,D,SE,Z
"3L,3R","D(GHgam, GAgam; AOcol, chri)",-0.027,0.0029,-9.2
"3L,3R","D(GHgam, GAgam; BFcol, chri)",-0.011,0.0031,-3.6
"3L,3R","D(GHgam, GAgam; GNcol, chri)",-0.009,0.0026,-3.4
"3L,3R","D(GHgam, GAgam; CIcol, chri)",-0.010,0.0027,-3.7
"3L,3R","D(GHgam, GAgam; GHcol, chri)",-0.011,0.0032,-3.4
"3L,3R","D(CMgam, GAgam; AOcol, chri)",-0.030,0.0029,-10.6
"3L,3R","D(CMgam, GAgam; BFcol, chri)",-0.017,0.0029,-5.9
"3L,3R","D(CMgam, GAgam; GNcol, chri)",-0.015,0.0024,-6.4
"3L,3R","D(CMgam, GAgam; CIcol, chri)",-0.016,0.0024,-6.6
"3L,3R","D(CMgam, GAgam; GHcol, chri)",-0.018,0.0030,-5.8


In [42]:
f4_analys(A, B, C, D, chroms=("3L", "3R"),
                regions=(slice(15000000, 41000000), slice(1000000, 37000000)))

chromosome,test,D,SE,Z
"3L,3R","D(GHgam, GAgam; AOcol, chri)",-0.027,0.0029,-9.2
"3L,3R","D(GHgam, GAgam; BFcol, chri)",-0.011,0.0031,-3.6
"3L,3R","D(CMgam, GAgam; AOcol, chri)",-0.030,0.0029,-10.6
"3L,3R","D(CMgam, GAgam; BFcol, chri)",-0.017,0.0029,-5.9
"3L,3R","D(BFgam, GAgam; AOcol, chri)",-0.028,0.0026,-10.8
"3L,3R","D(BFgam, GAgam; BFcol, chri)",-0.013,0.0023,-5.8
"3L,3R","D(GNgam, GAgam; AOcol, chri)",-0.030,0.0026,-11.2
"3L,3R","D(GNgam, GAgam; BFcol, chri)",-0.014,0.0022,-6.4
"3L,3R","D(GQgam, GAgam; AOcol, chri)",-0.015,0.0030,-5.1
"3L,3R","D(GQgam, GAgam; BFcol, chri)",-0.010,0.0030,-3.5


In [18]:
A = an_gam
B = an_na
C = an_col
D = 'chri'

In [19]:
f4_analys(A, B, C, D, chroms=("3L", "3R"),
                regions=(slice(15000000, 41000000), slice(1000000, 37000000)))

chromosome,test,D,SE,Z
"3L,3R","D(GHgam, KE; AOcol, chri)",-0.046,0.0082,-5.6
"3L,3R","D(GHgam, KE; BFcol, chri)",-0.049,0.0080,-6.2
"3L,3R","D(GHgam, KE; GNcol, chri)",-0.048,0.0077,-6.2
"3L,3R","D(GHgam, KE; CIcol, chri)",-0.053,0.0079,-6.7
"3L,3R","D(GHgam, KE; GHcol, chri)",-0.057,0.0084,-6.7
"3L,3R","D(GHgam, GM; AOcol, chri)",-0.003,0.0039,-0.8
"3L,3R","D(GHgam, GM; BFcol, chri)",-0.019,0.0037,-5.1
"3L,3R","D(GHgam, GM; GNcol, chri)",-0.039,0.0043,-9.1
"3L,3R","D(GHgam, GM; CIcol, chri)",-0.028,0.0035,-8.0
"3L,3R","D(GHgam, GM; GHcol, chri)",-0.027,0.0039,-7.1


In [38]:
def plot_test(chroms, As,Bs,Cs,Ds, blen=100000):
    # normalise inputs
    if not isinstance(chroms, (list, tuple)):
        chroms = [chroms]
    if not isinstance(As, (list, tuple)):
        As = [As]
    if not isinstance(Bs, (list, tuple)):
        Bs = [Bs]
    if not isinstance(Cs, (list, tuple)):
        Cs = [Cs]
    if not isinstance(Ds, (list, tuple)):
        Ds = [Ds]
        
    tbl = []
    for chrom in chroms:
        for A in As:
            for B in Bs:
                if A != B:
                    for C in Cs:
                        for D in Ds:
                            if C != D:
                                d, d_se, d_z, b1, _ = f4_analysis(chrom, A, B, C, D, blen=blen)
                                test = 'D(%s, %s; %s, %s)' % (A, B, C, D)
                                row = b1
                                tbl += [row]
    tbl=np.array(tbl)
    #print (tbl)
    fig, ax = plt.subplots(figsize=(14, 4))
    # median
    pos = callset_biallel[chrom]['variants']['POS'][:]
    windows = allel.moving_statistic(pos, statistic=lambda v: [v[0], v[-1]], size=blen)
    x = np.asarray(windows).mean(axis=1)    
    y = np.median(tbl, axis=0)
    plt.plot(x, y, lw=1, label = "median", color=palette[1]) 
    
    #min and max
    y5 = np.max(tbl, axis=0)
    y6 = np.min(tbl, axis=0)
    ax.fill_between(x, y5, y6,label = "min and max", color=palette[3], alpha=.4)
    
    # interquartile range
    y1 = np.percentile(tbl, 25, axis=0)
    y2 = np.percentile(tbl, 75, axis=0)
    ax.fill_between(x, y1, y2, label = "interquartile range", color=palette[0], alpha=.6)
    # 5-95 range
    y3 = np.percentile(tbl, 5, axis=0)
    y4 = np.percentile(tbl, 95, axis=0)
    ax.fill_between(x, y3, y4,label = "5-95 range", color=palette[2], alpha=.4)

    
    ax.set_title(('ABBA BABA Test between Acol populations vs %s and %s populations on chromosome %s') % (C,D, chrom))
    sns.despine(ax=ax, offset=10)
    ax.set_xlim(0, pos.max())
    ax.set_xlabel('position (bp)')
    #ax.set_ylim(0,0.018)
    sns.despine(ax=ax, offset=10)
    fig.tight_layout()
    # show a legend on the plot 
    plt.legend() 

    # function to show the plot 
    plt.show()

In [39]:
chromosomes = ['3L', '3R', '2L', '2R', 'X']

In [43]:
plot_test(A, B, C, D, '3R')

KeyError: '3'